In [1]:
from vtk import vtkUnstructuredGridReader
from vtk import vtkXMLUnstructuredGridReader
from vtk.util import numpy_support as VN
from vtk.util.numpy_support import vtk_to_numpy
import numpy as np
import matplotlib.pyplot as plt
import os

In [3]:
# A python code that plots the deviatoric ve_stress_xx and ve_stress_yy along a vertical profile near the 
# attachment point of the beam. It also plots the analytic solution of sigma_xx from Geodynamics 3rd edition
# Turcotte and Schubert, equation 3.86 (replace h/2 with y). For a sufficiently thin plate, sigma_yy should be 0
# The arguments of the function are:
#    file_path - the pathway to the .vtu file to be read in
#    rho_beam - the density of the beam set in the input file
#    rho_amb - the density of the ambient material surrounding the beam set in the input file
#    mu - the elastic shear modulus of the beam set in the input file
#    nu - poissons ratio, 0.5 for incompressible
#    g - gravity set in the input file, this should be negative
#    beamtop - the top of the beam composition set in the input file
#    beambot - the bottom of the beam composition set in the input file
#    L - the length of the beam composition set in the input file
#    x_spacing - the x distance between the base mesh WITHOUT refinements. This is needed to make sure
#                that the profile is taken along a point where there are actually mesh points
######################################################################################################
def stress_profile(file_path, rho_beam, rho_amb, mu, nu, g, beamtop, beambot, L, x_spacing):
    
    # read in the data from the .vtu file
    reader = vtkXMLUnstructuredGridReader()
    reader.SetFileName(file_path)
    reader.Update()
    data = reader.GetOutput()
    points = data.GetPoints()
    x = vtk_to_numpy(points.GetData())
    sigxx = vtk_to_numpy(data.GetPointData().GetArray('ve_stress_xx'))
    sigyy = vtk_to_numpy(data.GetPointData().GetArray('ve_stress_yy'))
    pressure = vtk_to_numpy(data.GetPointData().GetArray('p'))
    
    # Create arrays to store the values along a vertical profile near attachment point
    sigxx_prof = []
    sigyy_prof = []
    pressure_prof = []
    y_prof = []
    x_prof = 5 * x_spacing
    for n in range(len(sigxx)):
        if x[:, 0][n] == x_prof:
            if x[:, 1][n] not in y_prof:
                sigxx_prof.append(sigxx[n])
                sigyy_prof.append(sigyy[n])
                y_prof.append(x[:, 1][n])
                pressure_prof.append(pressure[n])
    sigxx_prof = np.array(sigxx_prof)
    sigyy_prof = np.array(sigyy_prof)
    y_prof = np.array(y_prof)
    pressure_prof = np.array(pressure_prof)
    ymax = max(y_prof)
    
    # Set Parameters for calculating the analytic solution for sigma_xx
    drho = abs(rho_beam - rho_amb)
    h = beamtop - beambot
    E = 2 * mu * (1 + nu)
    D = E * h**3 / (12 * (1 - nu**2))
    q = drho * g * h
    y_anal = np.linspace(-h/2, h/2, len(y_prof))
    sigxx_anal = -E / (1 - nu**2) * y_anal * q / (2 * D) * (x_prof**2 - 2*L*x_prof + L**2)
    
    # Want to subtract out the lithostatic pressure from ASPECTs stress to isolate only stress due to flexing beam
    pressure_lith = []
    for y_val in y_prof:
        pressure_lith.append(rho_amb * -g * (ymax - y_val))
    pressure_lith = np.array(pressure_lith)

    # Plot
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (20, 10))
    fig.suptitle('$\sigma_{xx}$ and $\sigma_{yy}$ Profile in Beam Near Attachment Point', fontsize = 40)
    ax1.scatter( (sigxx_prof - pressure_prof + pressure_lith) / 1e6, y_prof / 1e3, label = 'ASPECT', c = 'b')
    ax1.plot(sigxx_anal / 1e6, (y_anal + beambot + h/2) / 1e3, label = 'Analytic', c = 'r')
    ax1.set_title("$\sigma_{xx}$", fontsize = 30)
    ax1.legend(fontsize = 20)
    ax1.set_xlabel('Stress - MPa', fontsize = 20)
    ax1.set_ylabel('y Position - km', fontsize = 20)
    ax1.set_ylim((beambot - 0.5 * h) / 1e3, (beamtop + 0.5 * h) / 1e3)
    
    ax2.scatter( (sigyy_prof - pressure_prof + pressure_lith) / 1e6, y_prof / 1e3, label = 'ASPECT', c = 'b')
    ax2.set_title('$\sigma_{yy}$', fontsize = 30)
    ax2.legend(fontsize = 20)
    ax2.set_xlabel("Stress - MPa", fontsize = 20)
    ax2.set_ylabel("y position - km", fontsize = 20)
    ax2.set_ylim( (beambot - 0.5 * h) / 1e3, (beamtop + 0.5 * h) / 1e3 )